In [8]:
from google.colab import drive
import os
import io

from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
from httplib2 import Http
from oauth2client import file, client, tools
from email.mime.text import MIMEText
from base64 import urlsafe_b64encode

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.compose']

drive.mount('/content/gdrive')     #Mount google drive
proj_folder = 'gdrive/My Drive/RCCB/' #My folder in which all m files are stored
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = proj_folder+'RCCB-cred.json'  #Load credential files from proj_folder

gmail_cred_files = proj_folder+'gmail-credentials.json'

token_fl = proj_folder+'token.pickle'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def create_service():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(token_fl):
        with open(token_fl, 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(gmail_cred_files, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    
    # Call the Gmail API for Testing
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'])
            
    return service

In [0]:
def create_message(sender, to, subject, message_text):

    
    """Create a message for an email.
      Args:
        sender: Email address of the sender.
        to: Email address of the receiver.
        subject: The subject of the email message.
        message_text: The text of the email message.
      Returns:
        An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    encoded_message = urlsafe_b64encode(message.as_bytes())
    return {'raw': encoded_message.decode()}

In [0]:
def send_message(service, user_id, message):
  """Send an email message.
  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.
  Returns:
    Sent Message.
  """
  try:
    message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
    print('Message Id: %s' % message['id'])
    return message
  #except errors.HttpError, error:
  except:
    print('An error occurred: %s' % error)


In [11]:
serv = create_service()
SENDER = 'atulsyan850@gmail.com'
RECIPIENT = 'okansht@gmail.com'
SUBJECT = 'Test Message'
CONTENT = 'Hello, I do not know you!'

raw_msg = create_message(SENDER, RECIPIENT, SUBJECT, CONTENT)
send_message(serv, "me", raw_msg)

Labels:
CATEGORY_PERSONAL
CATEGORY_SOCIAL
Junk
CATEGORY_FORUMS
IMPORTANT
CATEGORY_UPDATES
CHAT
SENT
INBOX
TRASH
CATEGORY_PROMOTIONS
DRAFT
SPAM
STARRED
UNREAD
Unwanted
Message Id: 16c86b09cb6d72fd


{'id': '16c86b09cb6d72fd',
 'labelIds': ['SENT'],
 'threadId': '16c86b09cb6d72fd'}